In [39]:
import pandas as pd
import json

In [40]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [41]:
# similariy test
fuzz.ratio("Where can I begin.", \
    "– Where can I begin.")


95

##### Mistakes of correctness.csv
###### 1. Some pred_target's format and length are incorrect due to wrong conversion
###### 2. 'sent_id' of 'graph-test-correctness.csv' are all wrong due to wrong sorting process
###### 3. It's necessary to adjust prediction results (json files) manually in certain steps 

In [42]:
df = pd.read_csv('../semEval16/check_correctness/graph-test-correctness.csv', encoding='utf-8')
df[df.Target_matching == True]

,sent_id,text,Source,Target,Polar_expression,Polarity,pred_target correct,pred_expression correct,hasTarget,hasExpression,Target_matching
52,en_BlueRibbonSushi_478218171:1,"Best food, phenominal service",NaN,service,Best,Positive,T,T,T,T,True
53,en_BlueRibbonSushi_478218171:2,That was the last night in Vegas and was too l...,NaN,most of the restaurant,closing -,Positive,T,T,T,T,True
54,en_BlueRibbonSushi_478218171:2,Best Neighborhood Standby.,NaN,Neighborhood Standby.,Best,Positive,T,T,T,T,True
55,en_BlueRibbonSushi_478218171:3,Open & Cool Place with the Best Pizza and Coffee,NaN,Coffee,Best,Positive,T,T,T,T,True
56,en_BlueRibbonSushi_478218333:0,Mediocre food,NaN,food,Mediocre,Positive,T,T,T,T,True
...,...,...,...,...,...,...,...,...,...,...,...
388,en_StackRestaurant__Bar_478538239:19,Try the Chef's Choice for sushi as the smoked ...,NaN,the rolls,tasty.,Positive,T,T,T,T,True
389,en_StackRestaurant__Bar_478538239:19,The pizza is delicious and the salads are fant...,NaN,the salads,fantastic.,Positive,T,T,T,T,True
390,en_StackRestaurant__Bar_478538239:7,The wine list is incredible and extensive and ...,NaN,the staff,"very nice, good",Positive,T,T,T,T,True
391,en_StackRestaurant__Bar_478538239:9,Best Indian food in L.A.,NaN,food,Best,Positive,T,T,T,T,True


In [43]:
# open json file of training data or test data
with open('../semEval16/graph_parser/predictions/test-gold/opener_en_prediction.json') as f:
    json_data = json.load(f)

json_data[:10]

[{'sent_id': 'en_BlueRibbonSushi_478218171:0', 'text': 'Yum!', 'opinions': []},
 {'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard'},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard'}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source': [[], []],
    'Target': [['Green Tea creme brulee'], ['0:22']],


In [44]:
# run this cell only for test data
# adjust sent_id from test-correctness

c = 0
for i in range(len(df)):
    flag = False

    for j in json_data:
        if df.iloc[i]['text'] == j['text'] or fuzz.ratio(df.iloc[i]['text'], j['text']) >= 95:
            c += 1
            flag = True
            df.iloc[i,0] = j['sent_id'] # edit sent_id
            break
            
    if flag == False:
        print(i, df.iloc[i]['text'])

print(f'number of df rows: {len(df)}')
print(f'number of df rows with correct sent_id: {c}')

803 #NAME?
number of df rows: 848
number of df rows with correct sent_id: 847


In [45]:
# assign 'Matching' to json based on correctness.csv(df)

count = 0

for i in json_data:
    sid = i['sent_id'] 

    for j in i['opinions']:
        j['Matching'] = False

        if len(j['Target'][0]) > 0:
            tar = j['Target'][0][0]
            exp = j['Polar_expression'][0][0]
            df2 = df[df.sent_id == sid]

            for k in range(len(df2)):
                # match pairs of json and csv
                if (pd.isnull(df2.iloc[k]['Target']) == False) and (df2.iloc[k]['Target'] == tar) and (df2.iloc[k]['Polar_expression'] == exp):
                    # print(df2.iloc[k]['Target'], df2.iloc[k]['Polar_expression'])   
                    if (df2.iloc[k]['Target_matching'] == True):
                        j['Matching'] = True
                        count += 1
                        break
                            
        
print(f'number of true targets: {count}')

number of true targets: 336


In [47]:
# run this cell only for test data
# same pair but different sent_id

for i in range(len(df)):
    if (df.iloc[i]['text'] == 'Great Breakfast'):
        print(i)
        print(df.iloc[i])

161
sent_id                     en_SchoonerOrLater_479615721:0
text                                       Great Breakfast
Source                                                 NaN
Target                                           Breakfast
Polar_expression                                     Great
Polarity                                          Positive
pred_target correct                                      T
pred_expression correct                                  T
hasTarget                                                T
hasExpression                                            T
Target_matching                                       True
Name: 161, dtype: object
288
sent_id                     en_SchoonerOrLater_479615721:0
text                                       Great Breakfast
Source                                                 NaN
Target                                           Breakfast
Polar_expression                                     Great
Polarity               

In [50]:
# run this cell only for test data
# edit duplicated sent_id

df.iloc[288,0] = "en_SnoozeanAMEatery_479991844:0"

In [51]:
# deep copy original json file
import copy
json_list = copy.deepcopy(json_data)

In [52]:
# check true pairs and print missing true pairs in json

true_df = df[df.Target_matching == True]
wrong = 0
sid_true = []

for s in range(len(true_df)):
    flag = False

    for t in range(len(json_list)):
        for r in json_list[t]['opinions']:
            if (r['Matching']) and (true_df.iloc[s]['sent_id'] == json_list[t]['sent_id']):
                if  (true_df.iloc[s]['Target'] == r['Target'][0][0]) and (true_df.iloc[s]['Polar_expression'] == r['Polar_expression'][0][0]):
                    flag = True
                    break
        if flag == True:
            break


    if flag == False:
        wrong += 1
        sid_true.append(true_df.iloc[s]['sent_id'])
        print(true_df.iloc[s]['sent_id'] + ', ' + true_df.iloc[s]['Target'] + ', ' + true_df.iloc[s]['Polar_expression'])

print(f'number of missing pairs: {wrong}')

en_MiopostoCaffe_479702043:10, The pizzas, not huge
en_MiopostoCaffe_479702043:4, The pizzas, thin crust
en_SnoozeanAMEatery_479991844:0, Breakfast, Great
en_MiopostoCaffe_479702043:6, The pizzas, light
en_MiopostoCaffe_479702043:6, The pizzas, scrumptious.
number of missing pairs: 5


In [55]:
# correct 'Matching' results manually
for i in range(len(json_list)):
    if json_list[i]['sent_id'] in sid_true:
        for j in range(len(json_list[i]['opinions'])):
            print(i,j,json_list[i]['opinions'][j])

json_list[497]['opinions'][0]['Matching'] = True
json_list[557]['opinions'][1]['Matching'] = True
json_list[559]['opinions'][0]['Matching'] = True
json_list[559]['opinions'][1]['Matching'] = True
json_list[563]['opinions'][0]['Matching'] = True

497 0 {'Source': [[], []], 'Target': [['Breakfast'], ['6:15']], 'Polar_expression': [['Great'], ['0:5']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
557 0 {'Source': [[], []], 'Target': [['the menu offers'], ['31:46']], 'Polar_expression': [['toppings.'], ['78:87']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
557 1 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['thin crust'], ['16:26']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
559 0 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['scrumptious.'], ['26:38']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
559 1 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['light'], ['16:21']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
563 0 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['not 

In [56]:
# check numbe of true pairs

count = 0
for i in range(len(json_list)):
        for j in json_list[i]['opinions']:
            if j['Matching']:
                count += 1

print(f'number of true targets: {count}')

number of true targets: 341


In [58]:
# remove all false pairs

for i in range(len(json_list)):
    new_list = [j for j in json_list[i]['opinions'] if j['Matching']]
    json_list[i]['opinions'] = new_list
        
json_list[:10]

[{'sent_id': 'en_BlueRibbonSushi_478218171:0', 'text': 'Yum!', 'opinions': []},
 {'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source'

In [59]:
# remove all empty opinions

json_new = []

for i in range(len(json_list)):
    if len(json_list[i]['opinions']) > 0:
        json_new.append(json_list[i])

print(f'number of text: {len(json_new)}')

number of text: 250


In [60]:
# add key of dataset

for i in range(len(json_new)):
    json_new[i]['dataset'] = 's16+_test'

json_new

[{'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_test'},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_test'},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source': [[], []],
    'Target': [[

In [61]:
# convert to json file

with open('../semEval16_plus/s16+_test.json', 'w') as outfile:
    json.dump(json_new, outfile)